In [1]:
import pandas as pd
import datetime
import numpy as np
import mysql.connector
import json
import csv
import os
import matplotlib as plt
import matplotlib.pyplot as pyplot
import dataframe_image as dfi
import requests

### Downloading the most recent sales data directly from the Census Bureau website

In [2]:
# 1. define url path
url = "https://www.census.gov/retail/mrts/www/mrtssales92-present.xlsx"

# 2. download the data behind the URL
response = requests.get(url)

# 3. Open the response into a new file called mrtssales92-present.xlsx
open("mrtssales92-present.xlsx", "wb").write(response.content)

440409

### Reading the MRTS Data and Data Preparation

In [3]:
xlsx = pd.ExcelFile("mrtssales92-present.xlsx")

sheets_dict = pd.read_excel(xlsx)

sales_df = pd.DataFrame()

for sheet in reversed(xlsx.sheet_names):
    
    # Capture the 65 Adjusted business types from column B in the spreadsheet and place them into a dataframe
    categories = pd.read_excel(xlsx, sheet_name=sheet, header = 3, usecols ="B").iloc[2:67]

    # The first sheet, 2022, has 7 less columns than the others so it has an exception
    if sheet == '2022':
        sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:J").iloc[1:66]
    else:
        # Otherwise place the 65 rows of values from columns C through N into a dataframe
        sales_figs = pd.read_excel(xlsx, sheet_name=sheet, header = 4, usecols ="C:N").iloc[1:66]

    # Combine the dataframe containing the business types with the dataframe that holds their associated sales figures.
    reshaped_df = pd.concat([categories.reset_index(drop=True),sales_figs.reset_index(drop=True)], axis=1)
    sales_df = pd.concat([sales_df,reshaped_df.reset_index(drop=True)], axis=1)

In [4]:
sales_df.shape

(65, 399)

In [5]:
sales_df.head()

,Kind of Business,Jan. 1992,Feb. 1992,Mar. 1992,Apr. 1992,May 1992,Jun. 1992,Jul. 1992,Aug. 1992,Sep. 1992,...,Dec. 2021,Kind of Business,Jan. 2022,Feb. 2022,Mar. 2022,Apr. 2022,May 2022,Jun. 2022,Jul. 2022,Aug. 2022
0,"Retail and food services sales, total",146376,147079,159336,163669,170068,168663,169890,170364,164617,...,711673,"Retail and food services sales, total",587961,579139,681854,681357,705001,697935,690050,702729
1,Retail sales and food services excl motor vehi...,116565,115862,124200,127587,133608,130274,132076,134928,128734,...,589369,Retail sales and food services excl motor vehi...,474414,462148,541746,544061,572984,565376,562385,566838
2,Retail sales and food services excl gasoline s...,134277,135499,147064,151226,156619,155236,156016,156539,151477,...,659844,Retail sales and food services excl gasoline s...,538863,529564,617457,616581,632118,622594,615971,634752
3,Retail sales and food services excl motor vehi...,104466,104282,111928,115144,120159,116847,118202,121103,115594,...,537540,Retail sales and food services excl motor vehi...,425316,412573,477349,479285,500101,490035,488306,498861
4,"Retail sales, total",130683,131244,142488,147175,152420,151849,152586,152476,148158,...,632849,"Retail sales, total",518146,506400,597077,594426,614474,609933,600746,613416


In [17]:
sales_df = sales_df.drop(columns ='Kind of Business')
sales_df.head()

,Jan. 1992,Feb. 1992,Mar. 1992,Apr. 1992,May 1992,Jun. 1992,Jul. 1992,Aug. 1992,Sep. 1992,Oct. 1992,Nov. 1992,Dec. 1992,Jan. 1993,Feb. 1993,Mar. 1993,Apr. 1993,May 1993,Jun. 1993,Jul. 1993,Aug. 1993,Sep. 1993,Oct. 1993,Nov. 1993,Dec. 1993,Jan. 1994,Feb. 1994,Mar. 1994,Apr. 1994,May 1994,Jun. 1994,Jul. 1994,Aug. 1994,Sep. 1994,Oct. 1994,Nov. 1994,Dec. 1994,Jan. 1995,Feb. 1995,Mar. 1995,Apr. 1995,May 1995,Jun. 1995,Jul. 1995,Aug. 1995,Sep. 1995,Oct. 1995,Nov. 1995,Dec. 1995,Jan. 1996,Feb. 1996,Mar. 1996,Apr. 1996,May 1996,Jun. 1996,Jul. 1996,Aug. 1996,Sep. 1996,Oct. 1996,Nov. 1996,Dec. 1996,Jan. 1997,Feb. 1997,Mar. 1997,Apr. 1997,May 1997,Jun. 1997,Jul. 1997,Aug. 1997,Sep. 1997,Oct. 1997,Nov. 1997,Dec. 1997,Jan. 1998,Feb. 1998,Mar. 1998,Apr. 1998,May 1998,Jun. 1998,Jul. 1998,Aug. 1998,Sep. 1998,Oct. 1998,Nov. 1998,Dec. 1998,Jan. 1999,Feb. 1999,Mar. 1999,Apr. 1999,May 1999,Jun. 1999,Jul. 1999,Aug. 1999,Sep. 1999,Oct. 1999,Nov. 1999,Dec. 1999,Jan. 2000,Feb. 2000,Mar. 2000,Apr. 2000,May 2000,Jun. 2000,Jul. 2000,Aug. 2000,Sep. 2000,Oct. 2000,Nov. 2000,Dec. 2000,Jan. 2001,Feb. 2001,Mar. 2001,Apr. 2001,May 2001,Jun. 2001,Jul. 2001,Aug. 2001,Sep. 2001,Oct. 2001,Nov. 2001,Dec. 2001,Jan. 2002,Feb. 2002,Mar. 2002,Apr. 2002,May 2002,Jun. 2002,Jul. 2002,Aug. 2002,Sep. 2002,Oct. 2002,Nov. 2002,Dec. 2002,Jan. 2003,Feb. 2003,Mar. 2003,Apr. 2003,May 2003,Jun. 2003,Jul. 2003,Aug. 2003,Sep. 2003,Oct. 2003,Nov. 2003,Dec. 2003,Jan. 2004,Feb. 2004,Mar. 2004,Apr. 2004,May 2004,Jun. 2004,Jul. 2004,Aug. 2004,Sep. 2004,Oct. 2004,Nov. 2004,Dec. 2004,Jan. 2005,Feb. 2005,Mar. 2005,Apr. 2005,May 2005,Jun. 2005,Jul. 2005,Aug. 2005,Sep. 2005,Oct. 2005,Nov. 2005,Dec. 2005,Jan. 2006,Feb. 2006,Mar. 2006,Apr. 2006,May 2006,Jun. 2006,Jul. 2006,Aug. 2006,Sep. 2006,Oct. 2006,Nov. 2006,Dec. 2006,Jan. 2007,Feb. 2007,Mar. 2007,Apr. 2007,May 2007,Jun. 2007,Jul. 2007,Aug. 2007,Sep. 2007,Oct. 2007,Nov. 2007,Dec. 2007,Jan. 2008,Feb. 2008,Mar. 2008,Apr. 2008,May 2008,Jun. 2008,Jul. 2008,Aug. 2008,Sep. 2008,Oct. 2008,Nov. 2008,Dec. 2008,Jan. 2009,Feb. 2009,Mar. 2009,Apr. 2009,May 2009,Jun. 2009,Jul. 2009,Aug. 2009,Sep. 2009,Oct. 2009,Nov. 2009,Dec. 2009,Jan. 2010,Feb. 2010,Mar. 2010,Apr. 2010,May 2010,Jun. 2010,Jul. 2010,Aug. 2010,Sep. 2010,Oct. 2010,Nov. 2010,Dec. 2010,Jan. 2011,Feb. 2011,Mar. 2011,Apr. 2011,May 2011,Jun. 2011,Jul. 2011,Aug. 2011,Sep. 2011,Oct. 2011,Nov. 2011,Dec. 2011,Jan. 2012,Feb. 2012,Mar. 2012,Apr. 2012,May 2012,Jun. 2012,Jul. 2012,Aug. 2012,Sep. 2012,Oct. 2012,Nov. 2012,Dec. 2012,Jan. 2013,Feb. 2013,Mar. 2013,Apr. 2013,May 2013,Jun. 2013,Jul. 2013,Aug. 2013,Sep. 2013,Oct. 2013,Nov. 2013,Dec. 2013,Jan. 2014,Feb. 2014,Mar. 2014,Apr. 2014,May 2014,Jun. 2014,Jul. 2014,Aug. 2014,Sep. 2014,Oct. 2014,Nov. 2014,Dec. 2014,Jan. 2015,Feb. 2015,Mar. 2015,Apr. 2015,May 2015,Jun. 2015,Jul. 2015,Aug. 2015,Sep. 2015,Oct. 2015,Nov. 2015,Dec. 2015,Jan. 2016,Feb. 2016,Mar. 2016,Apr. 2016,May 2016,Jun. 2016,Jul. 2016,Aug. 2016,Sep. 2016,Oct. 2016,Nov. 2016,Dec. 2016,Jan. 2017,Feb. 2017,Mar. 2017,Apr. 2017,May 2017,Jun. 2017,Jul. 2017,Aug. 2017,Sep. 2017,Oct. 2017,Nov. 2017,Dec. 2017,Jan. 2018,Feb. 2018,Mar. 2018,Apr. 2018,May 2018,Jun. 2018,Jul. 2018,Aug. 2018,Sep. 2018,Oct. 2018,Nov. 2018,Dec. 2018,Jan. 2019,Feb. 2019,Mar. 2019,Apr. 2019,May 2019,Jun. 2019,Jul. 2019,Aug. 2019,Sep. 2019,Oct. 2019,Nov. 2019,Dec. 2019,Jan. 2020,Feb. 2020,Mar. 2020,Apr. 2020,May 2020,Jun. 2020,Jul. 2020,Aug. 2020,Sep. 2020,Oct. 2020,Nov. 2020,Dec. 2020,Jan. 2021,Feb. 2021,Mar. 2021,Apr. 2021,May 2021,Jun. 2021,Jul. 2021,Aug. 2021,Sep. 2021,Oct. 2021,Nov. 2021,Dec. 2021,Jan. 2022,Feb. 2022,Mar. 2022,Apr. 2022,May 2022,Jun. 2022,Jul. 2022,Aug. 2022
0,146376,147079,159336,163669,170068,168663,169890,170364,164617,173655,171547,208838,153221,150087,170439,176456,182231,181535,183682,183318,177406,182737,187443,224540,161349,162841,192319,189569,194927,197946,193355,202388,193954,197956,202520,241111,175344,172138,201279,196039,210478,211844,203411,214248,202122,204044,212190,247491,185019,19238

In [18]:
transposed_df = sales_df.transpose()
transposed_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
Jan. 1992,146376,116565,134277,104466,130683,100872,33906,29811,26788,25800,24056,1744,3023,7503,3846,2392,1454,728,(NA),3657,601,(NA),8964,7637,(NA),842,29589,27306,(NA),1509,7258,6358,12099,6938,4852,701,1873,1649,(NA),1206,796,3023,972,620,790,14996,10306,4818,5488,4690,2579,2111,3642,1446,788,658,371,6860,2692,1916,15693,1049,13325,6887,6438
Feb. 1992,147079,115862,135499,104282,131244,100027,35220,31217,28203,27031,25041,1990,3014,7398,3908,2460,1448,731,(NA),3490,576,(NA),9023,7567,(NA),818,28570,26223,(NA),1541,7150,6284,11580,7524,5159,658,1991,1864,(NA),1265,1021,2806,1100,585,539,16015,11251,5347,5904,4764,2615,2149,3902,1477,679,798,402,6059,2285,1627,15835,1026,13474,6937,6537
Mar. 1992,159336,124200,147064,111928,142488,107352,38731,35136,31684,30195,28018,2177,3452,7826,4157,2607,1550,851,(NA),3669,631,(NA),10608,8718,(NA),958,29682,27235,(NA),1597,7413,6420,12272,8475,6024,731,2403,2152,(NA),1463,912,3040,1214,643,535,17984,12917,6262,6655,5067,2838,2229,3755,1406,692,714,419,6297,2484,1537,16848,1074,14346,7245,7101
Apr. 1992,163669,127587,151226,115144,147175,111093,40548,36082,32547,30583,27982,2601,3535,7668,4141,2525,1616,899,(NA),3527,643,(NA),11630,9330,(NA),1073,30228,27588,(NA),1675,7531,6573,12443,9401,6682,816,2665,2438,(NA),1675,969,3150,1267,706,523,18872,13461,6417,7044,5411,2984,2427,4148,1457,656,801,393,6022,2506,1341,16494,1031,14065,7000,7065
May 1992,170068,133608,156619,120159,152420,115960,42213,36460,32883,31095,28924,2171,3577,7846,4275,2605,1670,856,(NA),3571,662,(NA),12327,9731,(NA),1232,31677,28883,(NA),1822,7488,6491,13449,9558,6768,856,2752,2412,(NA),1560,1171,3175,1293,660,552,20037,14286,6636,7650,5751,3257,2494,4600,1597,697,900,435,5803,2483,1163,17648,1047,15077,7532,7545


In [19]:
categories['Kind of Business'] = categories['Kind of Business'].str.replace("Retail sales and food services","RSFS",regex=True)
categories['Kind of Business'] = categories['Kind of Business'].str.replace("\(|\)|\.|,|'","",regex=True).str.replace("-| ","_",regex=True)

transposed_df.columns = categories['Kind of Business'].to_list()
transposed_df = transposed_df.replace(to_replace="\(S\)|\(NA\)",value=np.nan,regex=True)
transposed_df = transposed_df.where((pd.notnull(transposed_df)), 0)

transposed_df.head()

,Retail_and_food_services_sales_total,RSFS_excl_motor_vehicle_and_parts,RSFS_excl_gasoline_stations,RSFS_excl_motor_vehicle_and_parts_and_gasoline_stations,Retail_sales_total,Retail_sales_total_excl_motor_vehicle_and_parts_dealers,GAFO1,Motor_vehicle_and_parts_dealers,Automobile_and_other_motor_vehicle_dealers,Automobile_dealers,New_car_dealers,Used_car_dealers,Automotive_parts_acc_and_tire_stores,Furniture_home_furn_electronics_and_appliance_stores,Furniture_and_home_furnishings_stores,Furniture_stores,Home_furnishings_stores,Floor_covering_stores,All_other_home_furnishings_stores,Electronics_and_appliance_stores,Household_appliance_stores,Electronics_stores,Building_mat_and_garden_equip_and_supplies_dealers,Building_mat_and_supplies_dealers,Paint_and_wallpaper_stores,Hardware_stores,Food_and_beverage_stores,Grocery_stores,Supermarkets_and_other_grocery_except_convenience_stores,Beer_wine_and_liquor_stores,Health_and_personal_care_stores,Pharmacies_and_drug_stores,Gasoline_stations,Clothing_and_clothing_access_stores,Clothing_stores,Mens_clothing_stores,Womens_clothing_stores,Family_clothing_stores,Other_clothing_stores,Shoe_stores,Jewelry_stores,Sporting_goods_hobby_musical_instrument_and_book_stores,Sporting_goods_stores,Hobby_toy_and_game_stores,Book_stores,General_merchandise_stores,Department_stores,Department_storesexcl_discount_department_stores,Discount_dept_stores,Other_general_merchandise_stores,Warehouse_clubs_and_superstores,All_other_gen_merchandise_stores,Miscellaneous_store_retailers,Office_supplies_stationery_and_gift_stores,Office_supplies_and_stationery_stores,Gift_novelty_and_souvenir_stores,Used_merchandise_stores,Nonstore_retailers,Electronic_shopping_and_mail_order_houses,Fuel_dealers,Food_services_and_drinking_places,Drinking_places,Restaurants_and_other_eating_places,Full_service_restaurants,Limited_service_eating_places
Jan. 1992,146376.0,116565.0,134277.0,104466.0,130683.0,100872.0,33906.0,29811.0,26788.0,25800.0,24056.0,1744.0,3023.0,7503.0,3846.0,2392.0,1454.0,728.0,0.0,3657.0,601.0,0.0,8964.0,7637.0,0.0,842.0,29589.0,27306.0,0.0,1509.0,7258.0,6358.0,12099.0,6938.0,4852.0,701.0,1873.0,1649.0,0.0,1206.0,796.0,3023.0,972.0,620.0,790.0,14996.0,10306.0,4818.0,5488.0,4690.0,2579.0,2111.0,3642.0,1446.0,788.0,658.0,371.0,6860.0,2692.0,1916.0,15693.0,1049.0,13325.0,6887.0,6438.0
Feb. 1992,147079.0,115862.0,135499.0,104282.0,131244.0,100027.0,35220.0,31217.0,28203.0,27031.0,25041.0,1990.0,3014.0,7398.0,3908.0,2460.0,1448.0,731.0,0.0,3490.0,576.0,0.0,9023.0,7567.0,0.0,818.0,28570.0,26223.0,0.0,1541.0,7150.0,6284.0,11580.0,7524.0,5159.0,658.0,1991.0,1864.0,0.0,1265.0,1021.0,2806.0,1100.0,585.0,539.0,16015.0,11251.0,5347.0,5904.0,4764.0,2615.0,2149.0,3902.0,1477.0,679.0,798.0,402.0,6059.0,2285.0,1627.0,15835.0,1026.0,13474.0,6937.0,6537.0
Mar. 1992,159336.0,124200.0,147064.0,111928.0,142488.0,107352.0,38731.0,35136.0,31684.0,30195.0,28018.0,2177.0,3452.0,7826.0,4157.0,2607.0,1550.0,851.0,0.0,3669.0,631.0,0.0,10608.0,8718.0,0.0,958.0,29682.0,27235.0,0.0,1597.0,7413.0,6420.0,12272.0,8475.0,6024.0,731.0,2403.0,2152.0,0.0,1463.0,912.0,3040.0,1214.0,643.0,535.0,17984.0,12917.0,6262.0,6655.0,5067.0,2838.0,2229.0,3755.0,1406.0,692.0,714.0,419.0,6297.0,2484.0,1537.0,16848.0,1074.0,14346.0,7245.0,7101.0
Apr. 1992,163669.0,127587.0,151226.0,115144.0,147175.0,111093.0,40548.0,36082.0,32547.0,30583.0,27982.0,2601.0,3535.0,7668.0,4141.0,2525.0,1616.0,899.0,0.0,3527.0,643.0,0.0,11630.0,9330.0,0.0,1073.0,30228.0,27588.0,0.0,1675.0,7531.0,6573.0,12443.0,9401.0,6682.0,816.0,2665.0,2438.0,0.0,1675.0,969.0,3150.0,1267.0,706.0,523.0,18872.0,13461.0,6417.0,7044.0,5411.0,2984.0,2427.0,4148.0,1457.0,656.0,801.0,393.0,6022.0,2506.0,1341.0,16494.0,1031.0,14065.0,7000.0,7065.0
May 1992,170068.0,133608.0,156619.0,120159.0,152420.0,115960.0,42213.0,36460.0,32883.0,31095.0,28924.0,2171.0,3577.0,7846.0,4275.0,2605.0,1670.0,856.0,0.0,3571.0,662.0,0.0,12327.0,9731.0,0.0,1232.0,31677.0,28883.0,0.0,1822.0,7488.0,6491.0,13449.0,9558.0,6768.0,85

In [35]:
melted_sales_df = pd.melt(transposed_df, var_name="Kind of Business", value_name="Sales", ignore_index=False)
melted_sales_df.head(10)

,Kind of Business,Sales
Month,,
1992-01-01,Retail_and_food_services_sales_total,146376.0
1992-02-01,Retail_and_food_services_sales_total,147079.0
1992-03-01,Retail_and_food_services_sales_total,159336.0
1992-04-01,Retail_and_food_services_sales_total,163669.0
1992-05-01,Retail_and_food_services_sales_total,170068.0
1992-06-01,Retail_and_food_services_sales_total,168663.0
1992-07-01,Retail_and_food_services_sales_total,169890.0
1992-08-01,Retail_and_food_services_sales_total,170364.0
1992-09-01,Retail_and_food_services_sales_total,164617.0


In [36]:
melted_sales_df.reset_index(inplace=True)
melted_sales_df.head()

,Month,Kind of Business,Sales
0,1992-01-01,Retail_and_food_services_sales_total,146376.0
1,1992-02-01,Retail_and_food_services_sales_total,147079.0
2,1992-03-01,Retail_and_food_services_sales_total,159336.0
3,1992-04-01,Retail_and_food_services_sales_total,163669.0
4,1992-05-01,Retail_and_food_services_sales_total,170068.0


In [37]:
melted_sales_df['Month']=pd.to_datetime(melted_sales_df['Month'])
melted_sales_df.head()

,Month,Kind of Business,Sales
0,1992-01-01,Retail_and_food_services_sales_total,146376.0
1,1992-02-01,Retail_and_food_services_sales_total,147079.0
2,1992-03-01,Retail_and_food_services_sales_total,159336.0
3,1992-04-01,Retail_and_food_services_sales_total,163669.0
4,1992-05-01,Retail_and_food_services_sales_total,170068.0


In [38]:
melted_sales_df.shape

(23920, 3)

### Exploring Percentage Change

In [41]:
# transposed_df = transposed_df.set_index('Month')

percent_changes = transposed_df.pct_change()

percent_changes.head()

# pd.options.display.float_format = '{:.1%}'.format

,Retail_and_food_services_sales_total,RSFS_excl_motor_vehicle_and_parts,RSFS_excl_gasoline_stations,RSFS_excl_motor_vehicle_and_parts_and_gasoline_stations,Retail_sales_total,Retail_sales_total_excl_motor_vehicle_and_parts_dealers,GAFO1,Motor_vehicle_and_parts_dealers,Automobile_and_other_motor_vehicle_dealers,Automobile_dealers,New_car_dealers,Used_car_dealers,Automotive_parts_acc_and_tire_stores,Furniture_home_furn_electronics_and_appliance_stores,Furniture_and_home_furnishings_stores,Furniture_stores,Home_furnishings_stores,Floor_covering_stores,All_other_home_furnishings_stores,Electronics_and_appliance_stores,Household_appliance_stores,Electronics_stores,Building_mat_and_garden_equip_and_supplies_dealers,Building_mat_and_supplies_dealers,Paint_and_wallpaper_stores,Hardware_stores,Food_and_beverage_stores,Grocery_stores,Supermarkets_and_other_grocery_except_convenience_stores,Beer_wine_and_liquor_stores,Health_and_personal_care_stores,Pharmacies_and_drug_stores,Gasoline_stations,Clothing_and_clothing_access_stores,Clothing_stores,Mens_clothing_stores,Womens_clothing_stores,Family_clothing_stores,Other_clothing_stores,Shoe_stores,Jewelry_stores,Sporting_goods_hobby_musical_instrument_and_book_stores,Sporting_goods_stores,Hobby_toy_and_game_stores,Book_stores,General_merchandise_stores,Department_stores,Department_storesexcl_discount_department_stores,Discount_dept_stores,Other_general_merchandise_stores,Warehouse_clubs_and_superstores,All_other_gen_merchandise_stores,Miscellaneous_store_retailers,Office_supplies_stationery_and_gift_stores,Office_supplies_and_stationery_stores,Gift_novelty_and_souvenir_stores,Used_merchandise_stores,Nonstore_retailers,Electronic_shopping_and_mail_order_houses,Fuel_dealers,Food_services_and_drinking_places,Drinking_places,Restaurants_and_other_eating_places,Full_service_restaurants,Limited_service_eating_places
Month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1992-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-02-01,0.004803,-0.006031,0.009101,-0.001761,0.004293,-0.008377,0.038754,0.047164,0.052822,0.047713,0.040946,0.141055,-0.002977,-0.013994,0.016121,0.028428,-0.004127,0.004121,NaN,-0.045666,-0.041597,NaN,0.006582,-0.009166,NaN,-0.028504,-0.034438,-0.039662,NaN,0.021206,-0.014880,-0.011639,-0.042896,0.084462,0.063273,-0.061341,0.063001,0.130382,NaN,0.048922,0.282663,-0.071783,0.131687,-0.056452,-0.317722,0.067951,0.091694,0.109797,0.075802,0.015778,0.013959,0.018001,0.071389,0.021438,-0.138325,0.212766,0.083558,-0.116764,-0.151189,-0.150835,0.009049,-0.021926,0.011182,0.007260,0.015377
1992-03-01,0.083336,0.071965,0.085351,0.073320,0.085672,0.073230,0.099688,0.125541,0.123427,0.117051,0.118885,0.093970,0.145322,0.057853,0.063715,0.059756,0.070442,0.164159,NaN,0.051289,0.095486,NaN,0.175662,0.152108,NaN,0.171149,0.038922,0.038592,NaN,0.036340,0.036783,0.021642,0.059758,0.126396,0.167668,0.110942,0.206931,0.154506,NaN,0.156522,-0.106758,0.083393,0.103636,0.099145,-0.007421,0.122947,0.148076,0.171124,0.127202,0.063602,0.085277,0.037227,-0.037673,-0.048070,0.019146,-0.105263,0.042289,0.039280,0.087090,-0.055317,0.063972,0.046784,0.064717,0.044400,0.086278
1992-04-01,0.027194,0.027271,0.028301,0.028733,0.032894,0.034848,0.046913,0.026924,0.027238,0.012850,-0.001285,0.194763,0.024044,-0.020189,-0.003849,-0.031454,0.042581,0.056404,NaN,-0.038703,0.019017,NaN,0.096342,0.070200,NaN,0.120042,0.018395,0.012961,NaN,0.048842,0.015918,0.023832,0.013934,0.109263,0.109230,0.116279,0.109030,0.132900,NaN,0.144908,0.062500,0.036184,0.043657,0.097978,-0.022430,0.049377,0.042115,0.024752,0.058452,0.067890,0.051445,0.088829,0.104660,0.036273,-0.052023,0.121849,-0.062053,-0.043672,0.008857,-0.127521,-0.021011,-0.040037,-0.019587,-0.033816,-0.005070
1992-05-01,0.039097,0.

In [42]:
melted_pc_df = pd.melt(percent_changes, var_name="Kind of Business", value_name="Percent Change", ignore_index=False)
melted_pc_df.head(10)

,Kind of Business,Percent Change
Month,,
1992-01-01,Retail_and_food_services_sales_total,NaN
1992-02-01,Retail_and_food_services_sales_total,0.004803
1992-03-01,Retail_and_food_services_sales_total,0.083336
1992-04-01,Retail_and_food_services_sales_total,0.027194
1992-05-01,Retail_and_food_services_sales_total,0.039097
1992-06-01,Retail_and_food_services_sales_total,-0.008261
1992-07-01,Retail_and_food_services_sales_total,0.007275
1992-08-01,Retail_and_food_services_sales_total,0.002790
1992-09-01,Retail_and_food_services_sales_total,-0.033734


In [43]:
melted_pc_df.reset_index(inplace=True)
melted_pc_df.head()

,Month,Kind of Business,Percent Change
0,1992-01-01,Retail_and_food_services_sales_total,NaN
1,1992-02-01,Retail_and_food_services_sales_total,0.004803
2,1992-03-01,Retail_and_food_services_sales_total,0.083336
3,1992-04-01,Retail_and_food_services_sales_total,0.027194
4,1992-05-01,Retail_and_food_services_sales_total,0.039097


In [44]:
melted_pc_df['Month']=pd.to_datetime(melted_pc_df['Month'])
melted_pc_df.head()

,Month,Kind of Business,Percent Change
0,1992-01-01,Retail_and_food_services_sales_total,NaN
1,1992-02-01,Retail_and_food_services_sales_total,0.004803
2,1992-03-01,Retail_and_food_services_sales_total,0.083336
3,1992-04-01,Retail_and_food_services_sales_total,0.027194
4,1992-05-01,Retail_and_food_services_sales_total,0.039097


In [45]:
melted_pc_df.shape

(23920, 3)

## Reference

US Census Bureau: Monthly Retail Trade. https://www.census.gov/retail/index.html.